# General Imports

This notebook uses toast, tolteca, and toltectoast.

In [ ]:
import os

os.environ["TOAST_LOGLEVEL"] = "VERBOSE"

import numpy as np
import netCDF4 as nc
import wurlitzer
from astropy import units as u
import astropy.io.fits as af
from astropy.wcs import WCS
import matplotlib.pyplot as plt

import toast
import toast.ops
import toast.widgets
from toast.pixels_io_wcs import write_wcs_fits

import tolteca
from tolteca.datamodels import io as tio

import toltectoast as tt
from toltectoast import ops as ttops
from toltectoast import io as ttio
from toltectoast.vis import plot_noise_estim

In [ ]:
# Get our MPI world rank to use later when only one process needs to do
# plotting, etc.
comm, procs, rank = toast.get_world()

In [ ]:
# Output directory.  We are making some PDF plots, etc, and will save
# those to a directory relative to this notebook.
out_dir = "noise_estimation_outputs"
if rank == 0:
    if not os.path.isdir(out_dir):
        os.mkdir(out_dir)

In [ ]:
# Input data

# test2 sim
# raw_dir = "/home/kisner/data/toltec/simraw/test2/test2_simu"
# red_dir = "/home/kisner/data/toltec/simraw/test2/test2_redu/redu00/000001"
# apt_file = os.path.join(raw_dir, "apt_000001_000_0000_2022_04_08_16_08_56.ecsv")
# tele_file = os.path.join(raw_dir, "tel_000001_000_0000_2022_04_08_16_08_56.nc")
# proc_file = os.path.join(red_dir, "toltec_simu_science_timestream_000001.nc")

# test3 sim
raw_dir = "/home/kisner/data/toltec/simraw/test3/test3_simu"
red_dir = "/home/kisner/data/toltec/simraw/test3/test3_redu/redu00/000001"
apt_file = os.path.join(raw_dir, "apt_000001_000_0000_2022_04_08_16_43_59.ecsv")
tele_file = os.path.join(raw_dir, "tel_000001_000_0000_2022_04_08_16_43_59.nc")
proc_file = os.path.join(red_dir, "toltec_simu_science_timestream_000001.nc")

# Load Data

**NOTE:** You can change the `thinfp` setting to be something large (say 64) which will reduce the number of detectors to `1/thinfp`.  This is useful for debugging, but any conclusions should be checked with the full set of detectors.

In [ ]:
data = toast.Data()

In [ ]:
loader = ttops.Load(
    files=[(apt_file, tele_file, proc_file, None)],
    array="a1100",
    det_pointing="quats",
    thinfp=16,
    check_nans=False,
)

In [ ]:
loader.apply(data)
data.info()

# Noise Estimation

In [ ]:
estim = toast.ops.NoiseEstim(
    det_data="signal",
    out_model="noise_est",
    det_flag_mask=0,
    lagmax=1000,
    nbin_psd=64,
    nsum=1,
    naverage=60,
)
estim.apply(data)

In [ ]:
# Compute a 1/f fit to the noise estimates
noise_fit = toast.ops.FitNoiseModel(
    noise_model=estim.out_model,
    out_model="fit_noise",
)
noise_fit.apply(data)

In [ ]:
# Cut outliers
noise_cut = toast.ops.FlagNoiseFit(
    noise_model=noise_fit.out_model,
)
noise_cut.apply(data)

In [ ]:
ob = data.obs[0]
model_est = ob[estim.out_model]
model_fit = ob[noise_fit.out_model]

for det in ob.local_detectors:
    plot_noise_estim(
        os.path.join(out_dir, f"noise_estim_{det}.png"),  
        model_est.freq(det), 
        model_est.psd(det), 
        fit_freq=model_fit.freq(det),
        fit_psd=model_fit.psd(det),
        semilog=False,
    )